## WebLLM 网页版大语言模型例子
* 学习要点：
    1. 使用网络API提供的大语言模型
        * 使用 **WebLLM** 建构聊天机器人，WebLLM特性是可在[本机和离线环境中运作](https://web.dev/articles/ai-chatbot-webllm?hl=zh-tw)
        * 基本思路及代码：
            * WebLLM 主要使用 javascript (js) 的动态网页程式脚本使用
            * 本处代码使用 Python 语言进行打包运用
            * 本处代码使用 Python 的程序库，用于 UI 交互 及 信息可视化 的 panel
    2. 网页/网络 的 通讯基本难点在于对 非同步 IO （Asynchronous I/O ）的掌握
    3. **轻快**：使用轻量級的技术线，能较快取得大语言模型（档案较小），能在浏览器使用
        * 电子笔记环境：JupyterLite 是可以直接使用浏览器开启的轻量级的 JupyterLab
        * WebLLM： WebLLM 将开放式大型语言模型(LLM) 带入浏览器，让使用者只需开启网页就能直接执行 LLM，无需繁琐的安装步骤，涉及如 WebGPU 是的 新 W3C 标准
        * UI 交互：panelite 是可以直接使用浏览器开启的轻量级的 Panel
        * 大语言模型选择思考：档案大小、使用场景等等的考量，将需求及功能对齐。
* 下一步进阶：[Panel Chat Examples](https://holoviz-topics.github.io/panel-chat-examples/)

### 使用 Panelite 提示

<div class="alert alert-block alert-success">
<em>Panelite</em> is powered by young technologies like <a href="https://pyodide.org/en/stable/">Pyodide</a> and <a href="https://jupyterlite.readthedocs.io/en/latest/">Jupyterlite</a>. Some browsers may be poorly supported (e.g. mobile or 32-bit versions). If you experience issues, please <a href="https://github.com/holoviz/panel/issues">report them</a>.
</div>

### 代码：所需的 Python 库 
* asyncio： 非同步 IO （Asynchronous I/O ）的通讯，数据来回，就是 **有问有答** 的 底层的  **有来有回** 通讯交互基础
* panel： UI 交互 套件，提供如输入对话框，回应框，以及调参控制件的 UI 交互工具库
* param： Python 的参数化库，便于管理参数

In [1]:
import asyncio
import panel as pn
import param

from panel.custom import JSComponent, ESMEvent

pn.extension('mathjax', template='material')

#### 说明 API (应用 App 的 Interface )
此例展示如何用 Python 打包一外部 （javascript）套件 (指的是 [WebLLM](https://github.com/mlc-ai/web-llm)) 的 `JSComponent` ，并以 `ChatInterface` 做为 接入接出 的 接口 (Interface)。

* 📌(廖注) 这里涉及到 API 应用程序接口的基本概念及理解。

This example demonstrates how to wrap an external library (specifically [WebLLM](https://github.com/mlc-ai/web-llm)) as a `JSComponent` and interface it with the `ChatInterface`.

* 📌(廖注) 
    * MODELS 变量定义了给人看的名称（键），及其在 WebLLM 的登录全名。
    * class WebLLM(JSComponent) 将 JSComponent 打包成名为 WebLLM 的类别物件进行运用。
        * 包括 _esm 的 javascript 代码，交待了用 javascript 和 WebLLM 取得 模型在浏览器运用的沟通代码
        * 包括 menu(self) 的 UI 物件，有以下：
            * LLM模型的**选择**控件： pn.widgets.Select ... self.param.model
            * LLM模型的**调参**控件： pn.widgets.FloatSlider ... self.param.temperature, 
            * 其它UI：如按钮 Button, 进度 self.param.loading 等等

In [ ]:
MODELS = {
    'SmolLM (130MB)': 'SmolLM-135M-Instruct-q4f16_1-MLC',
    'TinyLlama-1.1B-Chat (675 MB)': 'TinyLlama-1.1B-Chat-v1.0-q4f16_1-MLC-1k',
    'Gemma-2b (2GB)': 'gemma-2-2b-it-q4f16_1-MLC',
    'Llama-3.2-3B-Instruct (2.2GB)': 'Llama-3.2-3B-Instruct-q4f16_1-MLC',
    'Mistral-7b-Instruct (5GB)': 'Mistral-7B-Instruct-v0.3-q4f16_1-MLC',
}

class WebLLM(JSComponent):

    loaded = param.Boolean(default=False, doc="""
        Whether the model is loaded.""")

    history = param.Integer(default=3)

    status = param.Dict(default={'text': '', 'progress': 0})

    load_model = param.Event()

    model = param.Selector(default='SmolLM-135M-Instruct-q4f16_1-MLC', objects=MODELS)

    running = param.Boolean(default=False, doc="""
        Whether the LLM is currently running.""")
    
    temperature = param.Number(default=1, bounds=(0, 2), doc="""
        Temperature of the model completions.""")

    _esm = """
    import * as webllm from "https://esm.run/@mlc-ai/web-llm";

    const engines = new Map()

    export async function render({ model }) {
      model.on("msg:custom", async (event) => {
        if (event.type === 'load') {
          if (!engines.has(model.model)) {
            const initProgressCallback = (status) => {
              model.status = status
            }
            const mlc = await webllm.CreateMLCEngine(
               model.model,
               {initProgressCallback}
            )
            engines.set(model.model, mlc)
          }
          model.loaded = true
        } else if (event.type === 'completion') {
          const engine = engines.get(model.model)
          if (engine == null) {
            model.send_msg({'finish_reason': 'error'})
          }
          const chunks = await engine.chat.completions.create({
            messages: event.messages,
            temperature: model.temperature ,
            stream: true,
          })
          model.running = true
          for await (const chunk of chunks) {
            if (!model.running) {
              break
            }
            model.send_msg(chunk.choices[0])
          }
        }
      })
    }
    """

    def __init__(self, **params):
        super().__init__(**params)
        if pn.state.location:
            pn.state.location.sync(self, {'model': 'model'})
        self._buffer = []

    @param.depends('load_model', watch=True)
    def _load_model(self):
        self.loading = True
        self._send_msg({'type': 'load'})

    @param.depends('loaded', watch=True)
    def _loaded(self):
        self.loading = False

    @param.depends('model', watch=True)
    def _update_load_model(self):
        self.loaded = False

    def _handle_msg(self, msg):
        if self.running:
            self._buffer.insert(0, msg)

    async def create_completion(self, msgs):
        self._send_msg({'type': 'completion', 'messages': msgs})
        while True:
            await asyncio.sleep(0.01)
            if not self._buffer:
                continue
            choice = self._buffer.pop()
            yield choice
            reason = choice['finish_reason']
            if reason == 'error':
                raise RuntimeError('Model not loaded')
            elif reason:
                return

    async def callback(self, contents: str, user: str):
        if not self.loaded:
            if self.loading:
                yield pn.pane.Markdown(
                    f'## `{self.model}`\n\n' + self.param.status.rx()['text']
                )
            else:
                yield 'Load the model'
            return
        self.running = False
        self._buffer.clear()
        message = ""
        async for chunk in self.create_completion([{'role': 'user', 'content': contents}]):
            message += chunk['delta'].get('content', '')
            yield message

    def menu(self):
        status = self.param.status.rx()
        return pn.Column(
            pn.widgets.Select.from_param(self.param.model, name='请选择一模型', sizing_mode='stretch_width'),
            pn.widgets.FloatSlider.from_param(self.param.temperature, name='请选择一温度值', sizing_mode='stretch_width'),
            pn.widgets.Button.from_param(
                self.param.load_model, sizing_mode='stretch_width',
                disabled=self.param.loaded.rx().rx.or_(self.param.loading)
            ),
            pn.indicators.Progress(
                value=(status['progress']*100).rx.pipe(int), visible=self.param.loading,
                sizing_mode='stretch_width'
            ),
            pn.pane.Markdown(status['text'], visible=self.param.loading)
        )

实现了  `WebLLM`  组件后，我们可以渲染WebLLM UI：

Having implemented the `WebLLM` component we can render the WebLLM UI:

In [ ]:
llm = WebLLM()

intro = pn.pane.Alert("""
`WebLLM` 运行大型语言模型，全在浏览器中。
首次访问应用程序时，可能需要一些时间下载模型并将其加载到内存中。
模型按大小（和功能）排序，例如，SmolLLM 下载速度非常快，但输出质量较差；
而 Mistral-7b 下载时间较长，但输出质量更高。
 """.replace('\n', ' '))

# intro = pn.pane.Alert("""
# `WebLLM` runs large-language models entirely in your browser.
# When visiting the application the first time the model has
# to be downloaded and loaded into memory, which may take 
# some time. Models are ordered by size (and capability),
# e.g. SmolLLM is very quick to download but produces poor
# quality output while Mistral-7b will take a while to
# download but produces much higher quality output.
# """.replace('\n', ' '))

pn.Column(
    llm.menu(),
    intro,
    llm
).servable(area='sidebar')

Column
    [0] Column
        [0] Select(name='Model', options=OrderedDict({'SmolLM (130M...]), sizing_mode='stretch_width', value='SmolLM-135M-Instruct-q4f1...)
        [1] FloatSlider(end=2, name='Temperature', sizing_mode='stretch_width', value=1)
        [2] Button(name='Load model', sizing_mode='stretch_width')
        [3] Progress(sizing_mode='stretch_width', value=0, visible=False)
        [4] Markdown(str, visible=False)
    [1] Alert(str, sizing_mode='stretch_width')
    [2] WebLLM()

并将其连接到 `ChatInterface`：

And connect it to a `ChatInterface`:

In [ ]:
chat_interface = pn.chat.ChatInterface(callback=llm.callback)
chat_interface.send(
    "读入模型，开始对谈。Load a model and start chatting.",
    user="System",
    respond=False,
)

llm.param.watch(lambda e: chat_interface.send(f'已成功读入， 开聊。 Loaded `{e.obj.model}`, start chatting!', user='System', respond=False), 'loaded')

pn.Row(chat_interface).servable(title='网页版大语言模型 WebLLM')

Row
    [0] ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(align='cen...}, _input_container=Row, _input_layout=Row, _placeholder=ChatMessage, _widgets={'ChatAreaInput': ChatArea...}, callback=<bound method W..., show_button_name=True, sizing_mode='stretch_width', widgets=[ChatAreaInput(css_classes...])
        [0] ChatMessage(object='Load a model and start chatting.', user='System', reactions=[])